In [1]:
%reload_ext autoreload
%autoreload 2
import sys, os
print(sys.executable)
module_path = os.path.abspath(os.path.join('..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)
# Verify kernal path
import numpy as np
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime
load_dotenv()

/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/bin/python


True

In [3]:
from src.inference import load_batch_of_features_from_store, get_future_predictions, load_model_from_registry, get_model_predictions
from src.data import transform_ts_data_into_features_target

ts_prices = load_batch_of_features_from_store()  # get historical feature-data from feature-store
ts_prices


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736
2024-08-19 22:09:22,906 WARNING: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead

2024-08-19 22:09:22,906 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.60s) 
2024-08-19 22:09:26,536 WARNING: VersionWarning: Incremented version to `173`.



,volume,vw_avr_price,open_price,close_price,high_price,low_price,num_transactions,datetime,sma_5,sma_20,ema_5,ema_20,vol_5,daily_return,price_diff,volume_sma_5,volume_change
341,60362338.0,165.8939,166.37,164.87,167.8100,164.2000,540017.0,2022-08-02 00:00:00+00:00,164.870,168.983000,164.870000,164.870000,0.000000,0.000000,0.00,60362338.0,0.000000
691,60362338.0,165.8939,166.37,164.87,167.8100,164.2000,540017.0,2022-08-03 00:00:00+00:00,164.870,168.983000,164.870000,164.870000,0.000000,0.000000,0.00,60362338.0,0.000000
363,60362338.0,165.8939,166.37,164.87,167.8100,164.2000,540017.0,2022-08-04 00:00:00+00:00,164.870,168.983000,164.870000,164.870000,0.000000,0.000000,0.00,60362338.0,0.000000
698,60362338.0,165.8939,166.37,164.87,167.8100,164.2000,540017.0,2022-08-05 00:00:00+00:00,164.870,168.983000,164.870000,164.870000,0.000000,0.000000,0.00,60362338.0,0.000000
352,60362338.0,165.8939,166.37,164.87,167.8100,164.2000,540017.0,2022-08-06 00:00:00+00:00,165.754,169.627000,164.870000,164.870000,1.948854,0.000000,0.00,62866611.4,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,42388970.0,225.4078,223.92,226.05,226.8271,223.6501,562568.0,2024-08-16 04:00:00+00:00,222.258,217.277333,222.668217,219.071721,3.318896,0.005918,1.33,41340047.2,-0.057212
755,42388970.0,225.4078,223.92,226.05,226.8271,223.6501,562568.0,2024-08-17 00:00:00+00:00,223.962,217.672500,223.795478,219.736319,2.322040,0.000000,0.00,42511207.4,0.000000
756,42388970.0,225.4078,223.92,226.05,226.8271,223.6501,562568.0,2024-08-18 00:00:00+00:00,224.918,218.063000,224.546986,220.337622,1.878209,0.000000,0.00,42532774.4,0.000000
758,42388970.0,225.4078,223.92,226.05,226.8271,223.6501,562568.0,2024-08-19 00:00:00+00:00,225.784,218.425500,225.047990,220.881658,0.594794,0.000000,0.00,42903435.6,0.000000


In [4]:
model = load_model_from_registry()  

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736
Connected. Call `.close()` to terminate connection gracefully.


In [5]:
# GET HISTORICAL DATA PREDICTIONS
n_previous_days = 12
step_size = 1
features, targets = transform_ts_data_into_features_target(ts_prices, n_previous_days, step_size) # convert ts-data from feature-store into features/targets for training

historical_predictions = get_model_predictions(model, features, ts_prices) 
historical_predictions = historical_predictions.reset_index(drop=False)  # Creates a new 'index' column
historical_predictions.rename(columns={'index': 'id'}, inplace=True)  # get a primary key for it 
historical_predictions

Number of sequences/examples: 746
Last val: 173.19
Target val: 173.19
FEATURES
(746, 13, 16)


,id,predicted_prices
0,0,172.872050
1,1,173.004692
2,2,173.783825
3,3,174.050739
4,4,172.487093
...,...,...
741,741,228.709975
742,742,227.387469
743,743,228.117402
744,744,227.486008


In [6]:
# GET FUTURE DATA PREDICTIONS
num_days = 10
future_predictions = get_future_predictions(num_days, ts_prices, model)
# future_predictions['datetime'] = pd.to_datetime(future_predictions['datetime'])
future_predictions

FUTURE DATES: ['2024-08-21', '2024-08-22', '2024-08-23', '2024-08-24', '2024-08-25', '2024-08-26', '2024-08-27', '2024-08-28', '2024-08-29', '2024-08-30']
2024-08-19 22:09:42,769 WARNING: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

2024-08-19 22:09:42,771 WARNING: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



,future_predicted_prices,datetime
0,142.473359,2024-08-21
1,226.410474,2024-08-22
2,225.370019,2024-08-23
3,226.037588,2024-08-24
4,225.465853,2024-08-25
5,224.455082,2024-08-26
6,224.301513,2024-08-27
7,224.806741,2024-08-28
8,224.301513,2024-08-29
9,224.301513,2024-08-30


In [7]:
# SAVE HISTORICAL PREDICTIONS 
from src.inference import get_feature_store
import src.config as config

# connect to the feature group, keep in mind these are the names and metadata from this feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name="model_predictions_historical2",
    version=1,
    description="Predictions generated by model on historical data",
    primary_key = ['id'],
    event_time='id',
)

feature_group.insert(historical_predictions, write_options={"wait_for_job": False})


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736
Connected. Call `.close()` to terminate connection gracefully.


Uploading Dataframe: 0.00% |          | Rows 0/746 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_historical2_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/835736/jobs/named/model_predictions_historical2_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x164566c00>, None)

In [8]:
# SAVE FUTURE PREDICTIONS 
from src.inference import get_feature_store
import src.config as config

# connect to the feature group, keep in mind these are the names and metadata from this feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name="model_predictions_future2",
    version=1,
    description="Predictions generated by model on future data",
    primary_key = ["datetime"],
    event_time='datetime',
)

feature_group.insert(future_predictions, write_options={"wait_for_job": False})


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736
Connected. Call `.close()` to terminate connection gracefully.


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_future2_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/835736/jobs/named/model_predictions_future2_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x164567ad0>, None)

In [28]:
# # LOAD HISTORICAL PREDICTIONS FROM STORE TO CHECK
# feature_store = get_feature_store()

# # Retrieve the feature group
# feature_group = feature_store.get_feature_group(
#     name="model_prediction_historical",
#     version=1
# )

# # Read the data from the feature group
# historical_predictions_df = feature_group.read()
# historical_predictions_df

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736


Connected. Call `.close()` to terminate connection gracefully.


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.47s) 


,predicted_prices,datetime
0,172.928325,2024-01-30 05:00:00+00:00
1,173.088198,2023-01-03 05:00:00+00:00
2,173.990845,2023-04-12 04:00:00+00:00
3,174.434349,2022-08-22 04:00:00+00:00
4,171.685435,2022-11-23 05:00:00+00:00
...,...,...
723,209.893192,2024-05-09 04:00:00+00:00
724,208.682686,2024-06-21 04:00:00+00:00
725,211.269024,2023-03-30 04:00:00+00:00
726,210.146128,2022-10-17 04:00:00+00:00


In [11]:
# LOAD FUTURE PREDICTIONS FROM STORE TO CHECK
feature_store = get_feature_store()

# Retrieve the feature group
feature_group = feature_store.get_feature_group(
    name="model_predictions_future2",
    version=1
)

# Read the data from the feature group
future_predictions_df = feature_group.read()
future_predictions_df  # make sure to sort

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.50s) 


,future_predicted_prices,datetime
0,141.135033,2024-08-12 00:00:00+00:00
1,143.540880,2024-08-19 00:00:00+00:00
2,138.243773,2024-08-10 00:00:00+00:00
3,141.316957,2024-08-14 00:00:00+00:00
4,141.864320,2024-08-16 00:00:00+00:00
5,144.473435,2024-08-18 00:00:00+00:00
6,140.188305,2024-08-11 00:00:00+00:00
7,142.390427,2024-08-17 00:00:00+00:00
8,141.495877,2024-08-13 00:00:00+00:00
9,140.444801,2024-08-15 00:00:00+00:00
